### 最終課題

In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse, urlunparse
import time
import collections

# 設定
BASE_URL = "https://www.musashino-u.ac.jp/"
SLEEP = 1.0  # 負荷軽減のための間隔（秒）
SESSION_HEADERS = {
    "User-Agent": "Mozilla/5.0 (compatible; SitemapBot/1.0; +https://example.org/bot)"
}

# 除外するファイル拡張子（静的資源）
EXCLUDE_EXTS = (
    ".jpg", ".jpeg", ".png", ".gif", ".svg", ".webp",
    ".css", ".js", ".pdf", ".zip", ".mp4", ".mp3",
    ".ico", ".ttf", ".woff", ".woff2"
)

def normalize_url(url: str) -> str:
    """フラグメントを除去し、スキーム・netloc・path・queryで正規化する（簡易）。"""
    parsed = urlparse(url)
    path = parsed.path or "/"
    if path != "/" and path.endswith("/"):
        path = path.rstrip("/")
    norm = urlunparse((parsed.scheme or "https", parsed.netloc, path, parsed.params, parsed.query, ""))
    return norm

def allowed_by_robots(base_url: str, target_path: str) -> bool:
    """robots.txtのDisallowを簡易チェックする。"""
    try:
        robots_url = urljoin(base_url, "/robots.txt")
        r = requests.get(robots_url, headers=SESSION_HEADERS, timeout=10)
        if r.status_code != 200:
            return True
        txt = r.text.splitlines()
        disallows = []
        user_agent = None
        for line in txt:
            line = line.strip()
            if not line or line.startswith("#"):
                continue
            if line.lower().startswith("user-agent:"):
                user_agent = line.split(":",1)[1].strip()
            elif line.lower().startswith("disallow:") and (user_agent == "*" or user_agent is None):
                path = line.split(":",1)[1].strip()
                if path:
                    disallows.append(path)
        for d in disallows:
            if target_path.startswith(d):
                return False
        return True
    except Exception:
        return True

def crawl_site(base_url=BASE_URL, sleep=SLEEP):
    session = requests.Session()
    session.headers.update(SESSION_HEADERS)

    base_netloc = urlparse(base_url).netloc
    start = normalize_url(base_url)

    if not allowed_by_robots(base_url, "/"):
        print("robots.txt によりクロールが制限されています。実行を中止します。")
        return {}

    q = collections.deque([start])
    visited = set()
    sitemap = {}

    while q:
        url = q.popleft()
        if url in visited:
            continue
        visited.add(url)

        try:
            time.sleep(sleep)
            resp = session.get(url, timeout=10)
            status = resp.status_code

            if status >= 400:
                print(f"HTTP {status} - {url} (スキップ)")
                continue

            ct = resp.headers.get("Content-Type", "")
            if "text/html" not in ct:
                print(f"非HTML ({ct}) をスキップ: {url}")
                continue

            resp.encoding = resp.apparent_encoding
            soup = BeautifulSoup(resp.text, "html.parser")

            title_tag = soup.find("title")
            title = title_tag.text.strip() if title_tag else "（タイトルなし）"
            sitemap[url] = title

            print(f"[{len(visited)}] 収集: {url} -> {title}")

            for a in soup.find_all("a", href=True):
                href = a["href"].strip()
                if href.startswith(("mailto:", "javascript:", "#")):
                    continue
                lower = href.lower()
                if any(lower.endswith(ext) for ext in EXCLUDE_EXTS):
                    continue

                new_url = urljoin(url, href)
                parsed = urlparse(new_url)
                if parsed.netloc != base_netloc:
                    continue

                norm = normalize_url(new_url)
                if norm not in visited and norm not in q:
                    q.append(norm)

        except Exception as e:
            print(f"例外が発生しました: {url} - {e}")
            continue

    print(f"\n完了：収集したページ数 = {len(sitemap)}")
    return sitemap

if __name__ == "__main__":
    site_map = crawl_site()

    # 全件を省略せずに表示
    print("\n--- サイトマップ一覧 ---")
    for url, title in site_map.items():
        print(f"{url} : {title}")

[1/200] 収集: https://www.musashino-u.ac.jp/ -> 武蔵野大学
[2/200] 収集: https://www.musashino-u.ac.jp/access.html -> 交通アクセス | 武蔵野大学
[3/200] 収集: https://www.musashino-u.ac.jp/admission/request.html -> 資料請求 | 入試情報 | 武蔵野大学
[4/200] 収集: https://www.musashino-u.ac.jp/contact.html -> お問い合わせ | 武蔵野大学
[5/200] 収集: https://www.musashino-u.ac.jp/prospective-students.html -> 武蔵野大学で学びたい方 | 武蔵野大学
[6/200] 収集: https://www.musashino-u.ac.jp/students.html -> 在学生の方 | 武蔵野大学
[7/200] 収集: https://www.musashino-u.ac.jp/alumni.html -> 卒業生の方 | 武蔵野大学
[8/200] 収集: https://www.musashino-u.ac.jp/parents.html -> 保護者の方 | 武蔵野大学
[9/200] 収集: https://www.musashino-u.ac.jp/business.html -> 企業・研究者の方 | 武蔵野大学
[10/200] 収集: https://www.musashino-u.ac.jp/guide -> 大学案内 | 武蔵野大学
[11/200] 収集: https://www.musashino-u.ac.jp/guide/profile -> 大学紹介 | 大学案内 | 武蔵野大学
[12/200] 収集: https://www.musashino-u.ac.jp/guide/activities -> 大学の取り組み | 大学案内 | 武蔵野大学
[13/200] 収集: https://www.musashino-u.ac.jp/guide/campus -> キャンパス | 大学案内 | 武蔵野大学
[14/200] 収集: https://